In [101]:
import gensim
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.font_manager as fm
from mpl_toolkits.mplot3d import Axes3D
import platform
from konlpy.tag import Okt
import re
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [2]:
# 한글 폰트 설정
system = platform.system()
if system == 'Darwin':  # macOS
    plt.rc('font', family='AppleGothic')
elif system == 'Windows':
    plt.rc('font', family='Malgun Gothic')
else:  # Linux
    available_fonts = [f.name for f in fm.fontManager.ttflist]
    for font in ['NanumGothic', 'NanumBarunGothic', 'UnDotum', 'UnBatang', 'Gulim']:
        if font in available_fonts:
            plt.rc('font', family=font)
            break
plt.rc('axes', unicode_minus=False)

# Load Kakao STS Corpus

In [3]:
from Korpora import Korpora
# 사용 가능한 말뭉치 목록 확인
Korpora.corpus_list()

{'kcbert': 'beomi@github 님이 만드신 KcBERT 학습데이터',
 'korean_chatbot_data': 'songys@github 님이 만드신 챗봇 문답 데이터',
 'korean_hate_speech': '{inmoonlight,warnikchow,beomi}@github 님이 만드신 혐오댓글데이터',
 'korean_parallel_koen_news': 'jungyeul@github 님이 만드신 병렬 말뭉치',
 'korean_petitions': 'lovit@github 님이 만드신 2017.08 ~ 2019.03 청와대 청원데이터',
 'kornli': 'KakaoBrain 에서 제공하는 Natural Language Inference (NLI) 데이터',
 'korsts': 'KakaoBrain 에서 제공하는 Semantic Textual Similarity (STS) 데이터',
 'kowikitext': 'lovit@github 님이 만드신 wikitext 형식의 한국어 위키피디아 데이터',
 'namuwikitext': 'lovit@github 님이 만드신 wikitext 형식의 나무위키 데이터',
 'naver_changwon_ner': '네이버 + 창원대 NER shared task data',
 'nsmc': 'e9t@github 님이 만드신 Naver sentiment movie corpus v1.0',
 'question_pair': 'songys@github 님이 만드신 질문쌍(Paired Question v.2)',
 'modu_news': '국립국어원에서 만든 모두의 말뭉치: 뉴스 말뭉치',
 'modu_messenger': '국립국어원에서 만든 모두의 말뭉치: 메신저 말뭉치',
 'modu_mp': '국립국어원에서 만든 모두의 말뭉치: 형태 분석 말뭉치',
 'modu_ne': '국립국어원에서 만든 모두의 말뭉치: 개체명 분석 말뭉치',
 'modu_spoken': '국립국어원에서 만든 모두의 말뭉치: 구어 

In [4]:
# Kopora가 지원하는 corpus 확인 및 다운로드
def listUpCorpus():
    for corpusName in Korpora.corpus_list():
        print(corpusName)
        try:
            corpus = Korpora.fetch(corpusName)

        except Exception as e:
            print(e)
    return 

#corpus_df = listUpCorpus()

In [5]:
# 다운로드 한 폴더 경로에서 corpus 목록 및 크기 정보 확인.
import os
from folder_size import get_folder_info
filePath = "/Users/bagjuhyeon/Korpora"

corpus_sizes = []
corpus_names = []
for corpus_name in os.listdir(filePath):
    corpus_path = os.path.join(filePath, corpus_name)
    
    if os.path.isdir(corpus_path):
        size = get_folder_info(corpus_path)['formatted_size']
        corpus_sizes.append(size)
        corpus_names.append(corpus_name)

corpusInfoDf = pd.DataFrame({'corpus_name': corpus_names, 'formatted_size': corpus_sizes})
corpusInfoDf.sort_values(by='formatted_size', ascending=True, inplace=True)

corpusInfoDf


'/Users/bagjuhyeon/Korpora/korean_parallel' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/korean_petitions' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/kowikitext' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/naver_changwon_ner' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/namuwikitext' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/question_pair' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/korsts' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/korean_hate_speech' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/kornli' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/kcbert' 폴더 크기 계산 중...
'/Users/bagjuhyeon/Korpora/nsmc' 폴더 크기 계산 중...


,corpus_name,formatted_size
6,korsts,1.53 MB
9,kcbert,11.83 GB
8,kornli,156.00 MB
3,naver_changwon_ner,16.16 MB
10,nsmc,18.62 MB
2,kowikitext,3.18 GB
4,namuwikitext,4.56 GB
7,korean_hate_speech,410.85 MB
0,korean_parallel,59.40 MB
1,korean_petitions,610.54 MB


# KoWiki

In [142]:
kowiki_path = os.path.join(filePath, "kowikitext")
os.listdir(kowiki_path)

['kowikitext_20200920.dev',
 '.DS_Store',
 'kowikitext_20200920.train.txt',
 'kowikitext_20200920.test',
 'kowikitext_20200920.train']

In [143]:
from read_korean_parallel import read_corpus_file
corpus = read_corpus_file(os.path.join(kowiki_path, "kowikitext_20200920.train"))
corpus

['= 분류:중화인민공화국의 외교부장 =',
 '',
 '외교부장',
 '외교부장',
 '',
 '',
 '',
 '= 분류:헝가리의 공원 =',
 '',
 '공원',
 '공원',
 '',
 '= 김세권 =',
 '',
 '김세권(1931년 ~ ,金世權) 은 제16대 서울고등검찰청 검사장을 역임한 법조인이다.',
 '',
 '= = 생애  = =',
 '',
 '1931년 서울시에서 태어나 경기중학교,1982년 4월 12일자 매일경제 서울고등학교,1981년 4월 25일자 동아일보 1956년 서울대학교 법학과를 나온 후 1956년 제8회 고등고시 사법과에서 합격하였다. 1958년 서울지방검찰청 검사에 임용되었다.',
 '',
 '김세권은 두산그룹 창업주인 박두병 딸인 박용언과 결혼했다.  김세권과 박용언은 아들은 1970년대 봉제업으로 성장한 태흥의 창업주 권태흥의 딸 권혜경과 결혼한 김형일 일경산업개발 부회장으로 1990년대 초반 대한민국에 게스·폴로 등을 수입해 유명세를 탔던 기업가다. 딸 김희정의 남편은 최원현 케이씨엘 대표변호사다.박승직 박두병 박용곤 박정원 재벌가 4대 33명 결혼 스토리',
 '',
 '대검찰청 차장으로 재직하던 1986년 2월 전국 검사장 회의를 주재하면서 "국법질서 확립을 위한 검찰의 과제"라는 주제로 토의를 하여 국법질서와 사회기강 확립, 경제도약의 전기가 될 수 있는 현재의 국내외 경제여건을 최대한 유지, 활용할 수 있도록 경제질서교란사범 엄단, 민생을 불안하게 하는 반윤리적인 강력사범에 대한 단호한 응징, 사회적 신분과 지위의 고하를 가리지 않는 엄정 공평한 법집행을 1986년도 검찰권 행사의 방향으로 정했다.1986년 2월 25일자 동아일보',
 '',
 '= = 경력  = =',
 '',
 '1958년 ~ 1959년 서울지방검찰청 검사',
 '1968년 법무부 출입국관리과장 겸 서울고등검찰청 검사',
 '1970년 ~ 1971년대검찰청 수사국 제3과장',
 '1971년 서울지방검찰청 동부지청 부장검사',
 '1975년 서울

In [11]:
len(corpus)


26794425

# Namuwiki

In [8]:
import os
namuwiki_path = os.path.join(filePath, "namuwikitext")
os.listdir(namuwiki_path)

['.DS_Store',
 'namuwikitext_20200302.train',
 'namuwikitext_20200302.test',
 'namuwikitext_20200302.dev']

In [9]:
corpus = read_corpus_file(os.path.join(namuwiki_path, "namuwikitext_20200302.train"))
corpus

['= 神様になった日 =',
 '',
 '= = 개요 = =',
 '',
 'REOL의 앨범 Σ의 일곱번째 트랙으로, 작곡의 원안을 레오루가 담당했다. 기가P는 후렴구의 멜로디를 위주로 작업했다고 하는데 가사와 잘 어울리는 우울하면서도 침착한 멜로디가 어우러져 깊은 여운을 남긴다.',
 '이번 앨범 내에서도 1, 2위를 다툴 정도로 어두운 가사로, 음악의 주제는 사별 ( 死別 ) . 신님이 된 날이라는 제목과 연관지어 생각해보면 꽤나 의미심장하다. 전 소절을 남겨진 사람과 떠난 사람 양쪽의 시점으로 해석 가능하다는 것도 특필할만한 점.',
 '',
 '= = 가사 = =',
 '',
 '= 하고로모코마치 =',
 '',
 '羽衣小町',
 "《아이돌 마스터 신데렐라 걸즈》에 등장하는 코바야카와 사에, 시오미 슈코로 이루어진 2인 유닛. 통칭 '슈사에'. 둘 다 교토가 고향이고 일본 전통 컨셉 아이돌이라는 공통점이 있다. 또한 이 둘은 아이돌을 하는데 있어서 집안과의 갈등을 포함하고 있다는 공통점도 있다. 사에는 엄격한 부모님의 반대를 무릅쓰고 상경, 슈코는 화과자 집 가업을 잇기 싫어서 가출 ( ... )",
 "모바마스에서 둘의 상호 아이돌 토크가 있고 '하고로모코마치'라는 유닛명을 달고 나와 여러 이벤트에서 출연하고 있다. 유닛명은 '날개옷 미녀'라는 뜻이다.",
 'MAGIC HOUR SP 9화에서 슈코와 사에 단 둘이서만 나와서 직접 유닛명을 언급했다.# 신데메이션 25화에서 사에, 슈코, 아즈키, 카코가 함께 행사를 꾸리는 장면이 스치듯 지나갔다. 신데렐라 걸즈 극장 애니 5화에서 사에, 슈코가 카오루를 데리고 꽃구경을 갔다.',
 '2016년 10월 15일에 열린 신데마스 4thLIVE SSA 공연에서 사에와 슈코의 성우가 青の一番星를 듀엣으로 부르는 무대가 있었다.',
 '데레스테에서 사에의 개인 커뮤에 슈코가 등장하거나, 사에의 SSR에 슈코가 같이 그려지는 등 친분 관계가 지속적으로 묘사되고 있다.',
 '2018년 1월 12일에 데레스테를 통해

In [10]:
len(corpus)

31235096

# Pre-processing

In [7]:
import pandas as pd
import json
import re
from tqdm import tqdm
#from multiprocessing import freeze_support
from fast_tokenizer import parallel_tokenize

def load_corpus(file_path, flag = None):
    """나무위키 덤프 파일을 로드합니다."""
    print(f"나무위키 코퍼스 로드 중: {file_path}")
    
    # JSON 형식인 경우
    if file_path.endswith('.json') or flag == 'json':
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    
    # 텍스트 파일인 경우
    elif file_path.endswith('.txt') or flag == None:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        return lines
    
    # 다른 형식 (예: jsonl)
    else:
        documents = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in tqdm(f, desc="파일 로드 중"):
                try:
                    doc = json.loads(line.strip())
                    documents.append(doc)
                except:
                    continue
        return documents

def extract_text(data):
    """나무위키 데이터에서 텍스트 내용을 추출합니다."""
    corpus = []
    print("데이터셋에서 텍스트 추출 중...")
    
    # 데이터 구조에 따라 적절히 수정 필요
    if isinstance(data, list) and isinstance(data[0], dict):
        # JSON 객체 리스트인 경우
        for doc in tqdm(data, desc="텍스트 추출 중"):
            if 'text' in doc:
                corpus.append(doc['text'])
            elif 'content' in doc:
                corpus.append(doc['content'])
    elif isinstance(data, list) and isinstance(data[0], str):
        # 문자열 리스트인 경우
        corpus = data
    
    # 문장 단위로 분리
    print("문장 단위로 분리 중...")
    corpus = [document for document in corpus if '.' in document]
    sentence_corpus = []
    for sentences in tqdm(corpus):
        for sentence in sentences.split('.'):
            if sentence != '':
                sentence_corpus.append(sentence + '.')
    
    return sentence_corpus

def clean_text(text):
    """위키 마크업을 제거하고 텍스트를 정제합니다."""
    # 파일 링크 제거
    text = re.sub(r'\[\[파일:.*?\]\]', '', text)
    
    # 링크 텍스트만 남기기
    text = re.sub(r'\[\[(.*?)\]\]', lambda m: m.group(1).split('|')[-1], text)
    
    # 각주 제거
    text = re.sub(r'<ref>.*?</ref>', '', text)
    text = re.sub(r'{{.*?}}', '', text)
    
    # 문법 요소 제거
    text = re.sub(r"'{2,}", '', text)  # 볼드체, 이탤릭체 제거
    text = re.sub(r'==+.*?==+', '', text)  # 제목 제거
    
    # HTML 태그 제거
    text = re.sub(r'<.*?>', '', text)
    
    # 여러 줄바꿈 정리
    text = re.sub(r'\n+', '\n', text)
    
    return text.strip()

def preprocessing_corpus(file_path, output_path=None, sample_size=None):
    """나무위키 코퍼스를 전처리합니다."""
    # 데이터 로드
    corpus = load_corpus(file_path)
    
    # 텍스트 추출
    corpus = extract_text(corpus)
    print(f"추출된 문서 수: {len(corpus)}")
    
    # 샘플링 (선택적)
    if sample_size and sample_size < len(corpus):
        import random
        corpus = random.sample(corpus, sample_size)
        print(f"샘플링된 문서 수: {len(corpus)}")
    
    # 텍스트 정제
    cleaned_corpus = []
    for doc in tqdm(corpus, desc="텍스트 정제 중"):
        cleaned_doc = clean_text(doc)
        if cleaned_doc:  # 빈 문서 제외
            # 문장 단위로 분리
            sentences = re.split(r'(?<=[.!?])\s+', cleaned_doc)
            cleaned_corpus.extend([s for s in sentences if len(s) > 10])  # 짧은 문장 제외
    
    print(f"정제 후 문장 수: {len(cleaned_corpus)}")
    
    # 결과 저장 (선택적)
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            for sentence in cleaned_corpus:
                f.write(sentence + '\n')
        print(f"전처리된 코퍼스 저장 완료: {output_path}")
    
    return cleaned_corpus

In [53]:
import os
namuwiki_path = os.path.join(filePath, "namuwikitext")
corpus = preprocessing_corpus(os.path.join(namuwiki_path, "namuwikitext_20200302.train"), output_path=os.path.join(namuwiki_path, "cleaned_namuwikitext_20200302.train"), sample_size=None)

나무위키 코퍼스 로드 중: /Users/bagjuhyeon/Korpora/namuwikitext/namuwikitext_20200302.train
데이터셋에서 텍스트 추출 중...
문장 단위로 분리 중...


 51%|█████     | 6092518/11929048 [00:04<00:04, 1349774.21it/s]


KeyboardInterrupt: 

In [54]:
import os
namuwiki_path = os.path.join(filePath, "namuwikitext")
sample_size = 1000000
corpus = preprocessing_corpus(os.path.join(namuwiki_path, "namuwikitext_20200302.train"), output_path=os.path.join(namuwiki_path, f"cleaned_namuwikitext_20200302_{sample_size}.train"), sample_size=sample_size)

나무위키 코퍼스 로드 중: /Users/bagjuhyeon/Korpora/namuwikitext/namuwikitext_20200302.train
데이터셋에서 텍스트 추출 중...
문장 단위로 분리 중...


100%|██████████| 11929048/11929048 [00:08<00:00, 1417281.27it/s]


추출된 문서 수: 46237147
샘플링된 문서 수: 1000000


텍스트 정제 중: 100%|██████████| 1000000/1000000 [00:04<00:00, 205107.04it/s]


정제 후 문장 수: 716296
전처리된 코퍼스 저장 완료: /Users/bagjuhyeon/Korpora/namuwikitext/cleaned_namuwikitext_20200302_1000000.train


In [55]:
corpus

['루머와는 다르게 계속해서 Channel Awesome에 영상을 올리는 것으로 보인다.',
 '비행능력이 없는 보행전차.',
 '조성주: 2016년 WESG 본선에서 조별리그를 포함하여 전승으로 결승에 진출했으나 전태양에게 3:4로 석패하면서 결승 전까지 전승, 결승에서도 풀세트 패배, 즉 이론상 이 대회에서 준우승자가 거둘 수 있는 승리란 승리는 모조리 다 거두며 최고 승률 준우승을 달성했다.',
 '하지만 이 세계선은 그녀가 올 리 없는 세계선인지라 나에의 바람은 지금 당장은 들어줄 수 없는 것이었고.',
 '1996년 12월 17일서해안고속도로 포승임시 나들목 ~ 안산 분기점 구간 개통과 함께 나들목으로 영업을 시작하였다.',
 '그의 팬인 여성 터치 풋볼팬 주장도 있는데 그와 비슷하게 머리를 하고다니며 기계치가 아닌데도 신을 따라한다며 일부러 기계를 부수고 다닌다고 한다.',
 "그러나, '박명수의 이상한 힌트'마저도 사실은 제작진이 사전에 준비한 것이었을 가능성이 매우 높다.",
 '단순 인지도만 봐도 일본의 애니메이션이라는 매체에 아주 조금이라도 관심이 있다면 본작의 존재를 모르는 사람이 거의 없을 정도.',
 '연인과 이별 후 빨래 같은 잡일을 통해서라도 그 아픔을 벗어나고자 하는 마음을 담고 있다.',
 '급기야 신나라에서 요서대윤 벼슬을 지내던 전담 ( 田譚 ) 이 이들을 추격하다가 오히려 죽고 말았는데, 이에 왕망은 그 허물을 고구려에게 돌렸다.',
 '실제로 일러스트에서도 유난히 큰 양 손이 돋보인다.',
 '그래서 크룹의 설계팀은 이를 모델로 하여 독일군의 표준구경인 8.',
 '2014년 방영한 11번째 ( 9대 ) 프리큐어 시리즈이며, 프리큐어 시리즈 10주년 기념작이다.',
 'K리그의 경우, 축구의 특성상 치어리더가 개입할 여건이 턱 없이 모자라며, 서포터즈들이 응원을 주도하기 때문에 치어리더 수요도 적다.',
 '각별한 애정을 보이는 등 플레이 기록이 몇 개월째 이어지는 중.',
 '모드린도 아자니가 에일리드 유적을 좋아한다고 

In [56]:
from fast_tokenizer import parallel_tokenize
tokenized_corpus = parallel_tokenize(corpus, pos_filter=None, batch_size=1000, n_jobs=None)

총 716296개 텍스트 처리 중... (9개 프로세스 사용)
배치 처리 중: 1~1000 / 716296


청크 처리 중: 100%|██████████| 10/10 [00:10<00:00,  1.08s/it]

배치 처리 중: 1001~2000 / 716296



청크 처리 중: 100%|██████████| 10/10 [00:06<00:00,  1.64it/s]

배치 처리 중: 2001~3000 / 716296


...

배치 처리 중: 715001~716000 / 716296



청크 처리 중: 100%|██████████| 10/10 [00:05<00:00,  1.98it/s]


배치 처리 중: 716001~716296 / 716296


청크 처리 중: 100%|██████████| 10/10 [00:04<00:00,  2.08it/s]

처리 완료: 716296개 텍스트 (4397.27초)


In [57]:
tokenized_corpus

[['루머', '다르', '계속', '아서', '영상', '올리', '보이', 'ㄴ다'],
 ['비행', '능력', '보행', '전차'],
 ['조성주',
  '본선',
  '에서',
  '조별',
  '리그',
  '포함',
  '전승',
  '결승',
  '진출',
  '으나',
  '전태양',
  '면서',
  '결승',
  '까지',
  '전승',
  '결승',
  '에서',
  '세트',
  '패배',
  '이론',
  '대회',
  '에서',
  '준우승',
  '거두',
  '승리',
  '승리',
  '모조리',
  '거두',
  '최고',
  '승률',
  '준우승',
  '달성'],
 ['하지만', '세계선', '그녀', '세계선', 'ㄴ지라', '바람', '지금', '당장', '들어주'],
 ['서해안고속도로', '포승', '임시', '나들', '안산 분기점', '구간', '개통', '함께', '나들', '영업', '시작'],
 ['여성',
  '터치',
  '풋볼',
  '주장',
  '는데',
  '비슷',
  '머리',
  '다니',
  '기계',
  '아니',
  'ㄴ데',
  '따르',
  'ㄴ다며',
  '일부러',
  '기계',
  '부수',
  '다니',
  'ㄴ다고',
  'ㄴ다'],
 ['그러나', '박명수', '이상', '힌트', '마저', '사실', '제작진', '사전', '준비', '가능', '매우'],
 ['단순',
  '인지도',
  '아도',
  '일본의 애니메이션',
  '라는',
  '매체',
  '아주',
  '조금',
  '라도',
  '관심',
  '다면',
  '작의',
  '존재',
  '모르',
  '사람',
  '거의',
  '정도'],
 ['연인', '이별', '빨래', '잡일', '통하', '아서라도', '아픔', '벗어나', '고자', '마음'],
 ['급기야',
  '신나라',
  '에서',
  '요서',
  '대유',
  '벼슬',
  '지내',
  '전담',
  '추격',
  '다가',

# Load KBS Corpus

In [13]:
subtitle = pd.read_csv("./corpus/kbs_corpus.csv")['subtitle']
sentence_df = []
target_token = ['(해설)', '(인터뷰)']
for item in subtitle:
    for sentence in item.split('\n'):
        if (target_token[0] in sentence) or (target_token[1] in sentence):
            sentence = sentence.replace(target_token[0], "")
            sentence = sentence.replace(target_token[1], "")
        sentence_df.append(sentence)
"""
sentence_df = pd.DataFrame({
    'sentence': sentence_df
})
"""
sentence_df

            

[' 한없이 높고도 넓은 이 품이 내내 그리웠습니다.',
 ' 아이고. 마고 할머니 잘 있었어?',
 ' 이 산을 지키고 있는 산신 할머니 입니다.',
 '깊고 장대한 백두대간의 끝자락.',
 '여기는 지리산 입니다.',
 ' 저기, 3개의 봉우리 보이십니까?',
 '네.',
 '여기는 그대로인데.',
 '저만 세월을 보낸 것 같습니다.',
 '언제나 변함없이 이 자리에서 우리를 기다리는 고향처럼 ',
 '푸근한 맛이 넘쳐서 그런 것 같습니다.',
 '이 깊어가는 가을.',
 '그 넉넉한 품을 내어주는 지리산 맛.',
 '여러분 함께 그 맛을 보시겠습니까.',
 '가시죠.',
 '낱알 하나가 얼마나 중요한데요.',
 '이렇게나 많이 달렸잖아요. ',
 '재첩이에요.',
 '골감이에요.',
 ' 바지런히 걷어들인 가을.',
 '1년 중 이때가 아니면 맛보기 힘든 풍성함 입니다.',
 '울긋불긋 맛있게 물든 밥상따라 만추의 지리산으로 떠납니다.',
 '지리산 중턱.',
 '마치 계단처럼 층층이 황금빛으로 물든 다랑이 논입니다.',
 '오늘은 이 마을 첫 나락 베는 날.',
 '벼를 척척 거두는 거는 농기계인데 어찌 농부 한 분이 그 뒤를 바삐 쫒아 다니시네요.',
 ' 논의 가장자리에 있는 벼는 벼 수확기계가 ',
 '들어가지 못해서 낫으로 베어주어야 해요.',
 '이곳은 모두 다랑논이어서 가장가지의 좁은 부분들은 ',
 '기계가 벼를 베지 못해요. ',
 '기계가 좋아서 벼를 벨 수 있지만 ',
 '전부 베지는 못해서 일일이 사람이 베어야 해요. ',
 '낱알 하나라도 버리면 안 되죠.',
 '이게 얼마나 중요한데요. ',
 '낱알이 정말 많이 달렸잖아요.',
 '벼 농사 짓는 사람들은 이 수확물 보는 보람으로 살죠.',
 ' 척박한 산 비탈을 맨 손으로 일구워 만든 계단식 논.',
 '어떻게든 쌀농사를 짓고자 했던 지리산 사람들의 간절함이 담겼으니 보람도 클 수 밖에요.',
 '오랜만에 추수의 손맛을 내는 도구까지 꺼냈습니다.',
 ' 기계 탈곡기가 나오기 전

In [15]:
# stop_words 파일 읽기
def load_stop_words(filepath):
    stop_words = []
    f = open(filepath, 'r')
    
    while True:
        line = f.readline()
        if not line: break
        stop_words.append(line.split("\t")[0])
    f.close()
    return stop_words

filepath = "./datasets/korean_stop_words.txt"
stop_words = load_stop_words(filepath)
stop_words

['이',
 '있',
 '하',
 '것',
 '들',
 '그',
 '되',
 '수',
 '이',
 '보',
 '않',
 '없',
 '나',
 '사람',
 '주',
 '아니',
 '등',
 '같',
 '우리',
 '때',
 '년',
 '가',
 '한',
 '지',
 '대하',
 '오',
 '말',
 '일',
 '그렇',
 '위하',
 '때문',
 '그것',
 '두',
 '말하',
 '알',
 '그러나',
 '받',
 '못하',
 '일',
 '그런',
 '또',
 '문제',
 '더',
 '사회',
 '많',
 '그리고',
 '좋',
 '크',
 '따르',
 '중',
 '나오',
 '가지',
 '씨',
 '시키',
 '만들',
 '지금',
 '생각하',
 '그러',
 '속',
 '하나',
 '집',
 '살',
 '모르',
 '적',
 '월',
 '데',
 '자신',
 '안',
 '어떤',
 '내',
 '내',
 '경우',
 '명',
 '생각',
 '시간',
 '그녀',
 '다시',
 '이런',
 '앞',
 '보이',
 '번',
 '나',
 '다른',
 '어떻',
 '여자',
 '개',
 '전',
 '들',
 '사실',
 '이렇',
 '점',
 '싶',
 '말',
 '정도',
 '좀',
 '원',
 '잘',
 '통하',
 '소리',
 '놓']

In [ ]:
import multiprocessing
from tqdm import tqdm
from functools import partial

# 형태소 분석기는 각 프로세스에서 개별적으로 초기화하도록 수정
def initialize_okt():
    from konlpy.tag import Okt
    return Okt()

# 불용어 집합으로 변환 (검색 속도 향상)
stop_words_set = set(stop_words)

# 정규식 패턴 미리 컴파일
korean_pattern = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]")

# 토큰화 함수 수정
def preprocess_single_text(text):
    # Okt 객체 초기화 (각 호출마다)
    okt = initialize_okt()
    
    # 정규화 (한글 외 문자 제거)
    text = korean_pattern.sub("", text)
    
    # 형태소 분석
    tokens = okt.morphs(text, stem=True)
    
    # 불용어 제거
    tokens = [word for word in tokens if word not in stop_words_set]
    
    return tokens

# 멀티프로세싱 풀 사용
def process_texts_in_parallel(corpus, num_processes=None):
    if num_processes is None:
        num_processes = multiprocessing.cpu_count() - 1  # 1개 코어는 여유롭게 남겨둠
    
    print(f"총 {len(corpus)}개 텍스트 처리 중... ({num_processes}개 프로세스 사용)")
    
    # 작은 배치로 나누어 처리 (메모리 관리)
    batch_size = 1000
    results = []
    
    # 전체 코퍼스를 배치로 나누어 처리
    for i in range(0, len(corpus), batch_size):
        batch = corpus[i:min(i+batch_size, len(corpus))]
        print(f"배치 처리 중: {i+1}~{i+len(batch)} / {len(corpus)}")
        
        # 멀티프로세싱 풀 생성 및 처리
        with multiprocessing.Pool(processes=num_processes) as pool:
            batch_results = list(tqdm(
                pool.imap(preprocess_single_text, batch),
                total=len(batch),
                desc="텍스트 처리 중"
            ))
            results.extend(batch_results)
    
    return results

# 병렬 처리 실행
tokenized_data = process_texts_in_parallel(corpus)
print(f"처리 완료: {len(tokenized_data)}개 텍스트")


총 31235096개 텍스트 처리 중... (9개 프로세스 사용)
배치 처리 중: 1~1000 / 31235096


Process SpawnPoolWorker-12:
Process SpawnPoolWorker-13:
Process SpawnPoolWorker-11:
Traceback (most recent call last):
Process SpawnPoolWorker-14:
Process SpawnPoolWorker-15:
Process SpawnPoolWorker-16:
Process SpawnPoolWorker-17:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/anaconda3/envs/w2v/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/anaconda3/envs/w2v/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/anaconda3/envs/w2v/lib/python3.12/multiprocessing/pool.py", line 114, in worker
    task = get()
           ^^^^^
  File "/opt/anaconda3/envs/w2v/lib/python3.12/multiprocessing/queues.py", line 389, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeE

In [58]:
# Word2Vec 모델 학습
model = Word2Vec(sentences=tokenized_corpus, 
                 sg=1,  # 1: skip-gram, 0: CBOW
                 vector_size=100,  # 벡터 차원
                 window=5,         # 컨텍스트 윈도우 크기
                 min_count=1,      # 최소 단어 빈도
                 workers=8)        # 병렬 처리 스레드 수

# 모델 저장
model.save('word2vec_namuwiki.model')

# 필요시 모델 로드
# model = Word2Vec.load('word2vec_model.model')


In [59]:
model.wv.index_to_key

['에서',
 'ㄴ다',
 '지만',
 '는데',
 '아서',
 '때문',
 '아니',
 '까지',
 '다고',
 '라고',
 '나오',
 '라는',
 '면서',
 '경우',
 '보이',
 '어서',
 '다는',
 '으며',
 '자신',
 '이후',
 '정도',
 '사람',
 '사용',
 '위하',
 '가능',
 '부터',
 '대하',
 '다른',
 '만들',
 '시키',
 '못하',
 '보다',
 '이나',
 '하지만',
 '그리고',
 '으나',
 '버리',
 '가지',
 '공격',
 '사실',
 '등장',
 '게임',
 '모습',
 '는다',
 '다가',
 '아야',
 '거나',
 '함께',
 '시작',
 '하나',
 '문제',
 '당하',
 '이상',
 '생각',
 '시간',
 '또한',
 '이름',
 '당시',
 '아도',
 '그렇',
 '이다',
 '가장',
 '처럼',
 '캐릭터',
 '상황',
 '따르',
 '결국',
 'ㄴ데',
 '다시',
 '다만',
 '으면',
 'ㄴ다는',
 '경기',
 '시즌',
 '모두',
 '상태',
 '많이',
 '능력',
 '모든',
 '상대',
 '현재',
 '존재',
 '물론',
 '매우',
 '일본',
 '그러나',
 '추가',
 '통하',
 '자체',
 '여기',
 '달리',
 '어야',
 '한국',
 '스키',
 'ㄴ다고',
 '이런',
 '이유',
 '같이',
 '들어가',
 '처음',
 '선수',
 '거의',
 '역시',
 '바로',
 '특히',
 '카드',
 '다면',
 '모르',
 '없이',
 '무기',
 '기록',
 '해당',
 '본인',
 '지역',
 '필요',
 '부분',
 '나가',
 'ㄴ지',
 '사건',
 '효과',
 '다음',
 '의하',
 '진행',
 '이야기',
 '자기',
 '기술',
 '세계',
 '는지',
 '전투',
 '다르',
 '대부분',
 '강하',
 '그냥',
 '시리즈',
 '여러',
 '마지막',
 '계속',
 '떨어지',
 '작품',
 '주인공',
 '동안',


In [60]:
model.corpus_count
model.corpus_total_words


7836322

In [20]:
def remove_stop_words(words, stop_words):
    return [word for word in words if word not in stop_words]

In [131]:
# 인터랙티브 3D 시각화
def interactive_3d_visualization(model, words=None, num_classes=None, topN=100):
    """
    인터랙티브 3D 시각화 (plotly 사용)
    
    Args:
        model: Word2Vec 모델
        words: 시각화할 단어 목록 (None이면 모델의 상위 단어 사용)
        topn: words가 None일 때 사용할 상위 단어 수
    """

    # 단어 목록 준비
    if words is None:
        available_words = len(model.wv.index_to_key)
        actual_topn = min(topN, available_words)
        words = [word for word in model.wv.index_to_key[:actual_topn]]
    else:
        words = [word for word in words if word in model.wv]
    
    # 단어 벡터 추출
    word_vectors = np.array([model.wv[word] for word in words])
    
    # perplexity 값 자동 조정
    n_samples = len(words)
    perplexity = min(max(5, n_samples // 4), n_samples - 1)


    # 클러스터링
    
    clustering = KMeans(n_clusters=num_classes, random_state=42)
    clustering.fit(word_vectors)
    """
    clustering = DBSCAN(eps=0.5, min_samples=1, metric='cosine')
    clustering.fit(word_vectors)
    """
    
    # 3차원 t-SNE 적용
    tsne = TSNE(n_components=3, random_state=42, max_iter=1000, perplexity=perplexity)
    embeddings_3d = tsne.fit_transform(word_vectors)
    
    # 데이터프레임 생성
    df = pd.DataFrame({
        'word': words,
        'x': embeddings_3d[:, 0],
        'y': embeddings_3d[:, 1],
        'z': embeddings_3d[:, 2],
        'cluster': clustering.labels_
    })
    
    # Plotly 3D 산점도 생성
    fig = px.scatter_3d(df, x='x', y='y', z='z', text='word',
                       title='Word2Vec 단어 임베딩 3D 시각화 (t-SNE)',
                       opacity=0.7, color='cluster')
    
    # 텍스트 레이블 추가
    fig.update_traces(textposition='top center')
    
    # 레이아웃 설정
    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        width=900,
        height=800,
        margin=dict(l=0, r=0, b=0, t=30)
    )
    
    fig.show()
    
    return fig, df

In [132]:
def w2v_visualization(model, words=None,num_classes=None, topN=1000):
    # 불용어 제거
    if words is None:
        words = remove_stop_words(model.wv.index_to_key[:topN], stop_words)
        
    # 인터랙티브 3D 시각화 (plotly 필요)
    _,df = interactive_3d_visualization(model, words=words, num_classes=num_classes)        
    return df

In [136]:
#model.wv.index_to_key

double_words = [
    {
        'keyword':['편도'],
        'related1':['길', '왕복', '왕반', '왕환'],
        'related2':['편도샘', '기관']
    },
    {
        'keyword':['이성'],
        'related1':['성', '동성', '여성', '여자', '남자', '남성', '연애', '사랑'],
        'related2':['참', '거짓', '판단','논리']
    },
    {
        'keyword':['이전'],
        'related1':['이동', '장소', '주소', '전택', '이사', '이주'],
        'related2':['이후','앞', '뒤', '요전', '시간', '전']
    },
    {
        'keyword':['자원'],
        'related1':['경제', '광물', '산림', '수산', '농업', '가스'],
        'related2':['스스로', '지원']
    },
    {
        'keyword':['사과'],
        'related1':['바나나', '오렌지', '배','참외', '수박', '딸기'],
        'related2':['미안','죄송', '죄송해', '용서', '잘못', '빌다']
    },
    {
        'keyword':['의사'],
        'related1':['가운', '병원', '간호사', '치과', '마취과', '수술'],
        'related2':['결정', '사고', '생각', '관념', '의사결정']
    },
    {
        'keyword':['양식'],
        'related1':['일식', '중식', '한식', '양식', '분식'],
        'related2':['모양', '형식', '형태', '형상', '형태'],
        'related3':['번식', '육성', '해조', '버섯'],
        'related4':['조리', '요리', '조리법', '조리방법', '조리법','식사']
    }
]

def taregt_words_generator(double_words, itemNum):
    """
    target_words = []
    for item in double_words:
        target_words.extend(item['keyword'] + item['related1'] + item['related2'])
    """
    target_words = []
    target_classes = -1
    for _, item in double_words[itemNum].items():
        target_words.extend(item)
        target_classes += 1

    return target_words, target_classes


target_words, target_classes = taregt_words_generator(double_words, itemNum=4)
len(target_words)

13

In [137]:
target_words, target_classes

(['사과',
  '바나나',
  '오렌지',
  '배',
  '참외',
  '수박',
  '딸기',
  '미안',
  '죄송',
  '죄송해',
  '용서',
  '잘못',
  '빌다'],
 2)

In [140]:
df = w2v_visualization(model, words=target_words, num_classes=target_classes)
#df = w2v_visualization(model)

In [141]:
df

,word,x,y,z,cluster
0,사과,-72.852829,-145.345917,61.048244,1
1,바나나,-136.455429,45.943195,-56.861870,0
2,오렌지,38.824490,146.041443,-131.394958,0
3,참외,142.111923,145.621964,15.986353,0
4,수박,-122.966255,-24.099110,367.275208,0
5,딸기,-96.256378,58.376961,-233.466705,0
6,미안,-24.430824,40.893860,154.729202,1
7,죄송,80.482010,-65.401550,75.303490,1
8,용서,-220.332199,-22.962263,144.822021,1
9,잘못,34.591999,-314.374756,-41.033669,0


In [53]:
#df[df['cluster'] == 3].drop('word', axis=1).corr()
df[df['cluster'] == 3]


,word,x,y,z,cluster
0,하다,-4.332164,1.270341,-12.356547,3
1,을,1.787266,-0.680015,-13.955473,3
2,에,6.190638,-13.832748,-20.716131,3
3,있다,-7.877423,-2.396724,-7.543454,3
4,를,14.644489,-10.868186,-27.198025,3
...,...,...,...,...,...
986,남자,-24.715105,13.841858,23.416382,3
989,남아,17.088943,6.235876,8.287522,3
993,정리,-10.160050,7.170446,21.658693,3
994,외,-7.732765,-23.921223,-25.799862,3


In [46]:
# 필요한 라이브러리 임포트
import torch
from transformers import BertModel
import numpy as np
import matplotlib.pyplot as plt
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', sp_model_kwargs={'nbest_size': -1, 'alpha': 0.6, 'enable_sampling': True})
tokenizer.encode("한국어 모델을 공유합니다.")


model = BertModel.from_pretrained('skt/kobert-base-v1')
text = "한국어 모델을 공유합니다."
inputs = tokenizer.batch_encode_plus([text])
out = model(input_ids = torch.tensor(inputs['input_ids']),
              attention_mask = torch.tensor(inputs['attention_mask']))
out.pooler_output.shape

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


torch.Size([1, 768])

In [3]:
out

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.1554, -0.0150,  0.3673,  ..., -0.0094,  0.1067,  0.0844],
         [ 0.1229, -0.3236, -0.0669,  ..., -0.4487, -0.1753, -0.2302],
         [ 0.1008, -0.3885, -0.1219,  ..., -0.2129, -0.0330, -0.1708],
         ...,
         [-0.0700,  0.1174, -0.1385,  ..., -0.0272,  0.5141, -0.0093],
         [-0.0605, -0.2776,  0.4285,  ..., -0.2879,  0.5493,  0.0797],
         [-0.1537, -0.1819, -0.1994,  ..., -0.2862,  0.0133,  0.0783]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-1.5197e-02,  1.2525e-02, -2.0906e-02, -4.9571e-02, -9.7731e-01,
          9.9156e-01, -6.7530e-01,  5.5347e-02,  2.3231e-02,  4.5099e-03,
         -5.1811e-01, -3.6802e-02, -4.0091e-02, -7.1836e-02, -2.3317e-02,
          8.3534e-01, -9.9642e-01,  5.2745e-02,  2.5370e-03, -8.5500e-02,
          1.1586e-01, -4.6590e-02, -2.1022e-02,  5.6610e-01,  3.4027e-02,
          7.5982e-01, -9.3067e-01, -6.7136e-02, -9.6185e-01, -1.712

In [47]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
def get_word_embeddings(words, model, tokenizer):
    """
    단어 리스트에서 임베딩을 추출합니다.
    
    Args:
        words: 임베딩을 추출할 단어 리스트
        model: 사전 훈련된 모델
        tokenizer: 모델에 맞는 토크나이저
        
    Returns:
        단어별 임베딩 (numpy 배열)
    """
    model.eval()
    word_embeddings = {}
    
    for word in words:
        # 토큰화
        inputs = tokenizer(word, return_tensors="pt")
        
        # 그래디언트 계산 없이 순전파
        with torch.no_grad():
            outputs = model(**inputs)
        
        # 단어 토큰의 임베딩 추출 (첫 번째 토큰 사용)
        # 서브워드 토크나이징으로 인해 하나의 단어가 여러 토큰으로 나뉠 수 있음
        tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        token_embeddings = outputs.last_hidden_state[0].numpy()
        
        # 첫 번째 토큰(서브워드)의 임베딩만 사용
        # [CLS] 토큰을 제외하고 실제 단어의 첫 번째 토큰은 인덱스 1에 위치
        word_embeddings[word] = token_embeddings[1]
    
    return word_embeddings

# 테스트할 한국어 단어들
words = ["한국", "서울", "인공지능", "컴퓨터", "프로그래밍", "음식", "문화", "기술"]

# 단어 임베딩 추출
word_embeddings = get_word_embeddings(target_words, model, tokenizer)

# 단어 임베딩 시각화 (2D PCA)
word_vectors = np.array([word_embeddings[word] for word in words])
pca = PCA(n_components=2)
word_embeddings_2d = pca.fit_transform(word_vectors)

plt.figure(figsize=(10, 8))
plt.scatter(word_embeddings_2d[:, 0], word_embeddings_2d[:, 1], marker='o')

for i, word in enumerate(words):
    plt.annotate(word, (word_embeddings_2d[i, 0], word_embeddings_2d[i, 1]), fontsize=12)

plt.title('KoBERT 단어 임베딩 PCA 시각화')
plt.xlabel('주성분 1')
plt.ylabel('주성분 2')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


# 단어 간 유사도 계산
word_similarity = cosine_similarity(word_vectors)

# 각 단어 쌍의 유사도 출력
print("단어 쌍 유사도:")
for i in range(len(words)):
    for j in range(i+1, len(words)):
       # print(f"'{words[i]}'와(과) '{words[j]}'의 유사도: {word_similarity[i][j]:.4f}")
       continue


KeyError: '서울'

In [45]:
word_embeddings

{'한국': array([ 5.20402193e-01, -1.85463399e-01, -8.17593336e-02, -5.23415804e+00,
         1.88694030e-01, -8.77131373e-02,  9.83812883e-02,  1.69055313e-01,
        -4.93674487e-01, -2.28505448e-01,  1.60057575e-01, -3.84428978e-01,
        -2.41844356e-01,  2.30651125e-01,  5.57950325e-02,  3.38617235e-01,
         1.65194035e-01,  4.07184027e-02, -6.69143438e-01,  1.77453548e-01,
        -4.34816390e-01,  6.51478618e-02,  7.15353414e-02,  4.58431870e-01,
        -1.04116373e-01,  8.23137015e-02, -9.38909829e-01, -2.62598217e-01,
        -7.97869042e-02, -1.71057209e-01,  1.57653898e-01,  5.90604171e-02,
        -6.58280134e-01, -2.41766289e-01,  5.00864573e-02, -1.62351251e-01,
         4.57898349e-01,  3.90168019e-02, -1.54968858e-01, -3.27769846e-01,
         8.20708424e-02, -3.65826860e-02, -1.08149052e-01,  4.38390285e-01,
        -7.15987757e-02, -3.23161483e-02,  2.28666455e-01,  3.46958227e-02,
         1.58328950e-01,  1.40993848e-01,  3.05026948e-01,  3.13351572e-01,
      